In [1]:
import pandas as pd



df = pd.read_csv("../nasdaq_exteral_data.csv", error_bad_lines=False, warn_bad_lines=False, engine='python', encoding='utf-8')




/var/folders/34/4m3rx1zd32dgvl3g8t313g900000gn/T/ipykernel_34569/1708142390.py:5: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv("../nasdaq_exteral_data.csv", error_bad_lines=False, warn_bad_lines=False, engine='python', encoding='utf-8')
/var/folders/34/4m3rx1zd32dgvl3g8t313g900000gn/T/ipykernel_34569/1708142390.py:5: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv("../nasdaq_exteral_data.csv", error_bad_lines=False, warn_bad_lines=False, engine='python', encoding='utf-8')


In [6]:
import os
import pandas as pd

df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

news = df[['Date', 'Article_title', 'Stock_symbol', 'Article']]


news = news.dropna()

stock_price_dir = "../full_history"

stock_price_dfs = []

for file in os.listdir(stock_price_dir):
    symbol = file.replace(".csv", "")
    df = pd.read_csv(os.path.join(stock_price_dir, file))
    df['Stock_symbol'] = symbol
    stock_price_dfs.append(df)

stock_df = pd.concat(stock_price_dfs, ignore_index=True)

stock_df["date"] = pd.to_datetime(stock_df["date"], errors='coerce')

print(stock_df.head())

KeyError: 'Date'

In [5]:
news['Date'] = pd.to_datetime(news['Date'], errors='coerce')
stock_df['date'] = pd.to_datetime(stock_df['date'], errors='coerce')

stock_df = stock_df.rename(columns={'date' : 'Date'})


merged_df = pd.merge_asof(
    news.sort_values(["Stock_symbol", "Date"]),
    stock_df.sort_values(["Stock_symbol", "Date"]),
    by="Stock_symbol",
    left_on="Date",
    right_on="Date",
    direction="backward"  # Get the most recent stock price before the news
)

print(merged_df.head())

KeyError: 'date'

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch
import re

tokenizer = BertTokenizer.from_pretrained("yiyanghkust/finbert-tone")

model = BertForSequenceClassification.from_pretrained("yiyanghkust/finbert-tone")


def get_sent(text):
    text = text.lower()
    text = re.sub(r"http\S+|www\S+|https\S+", "", text, flags=re.MULTILINE)
    text = re.sub(r"[^a-zA-Z0-9\s]", "", text)
    text = re.sub(r"\s+", " ", text).strip()
    inputs = tokenizer(text, return_tensors ="pt", truncation = True)
    outputs = model(**inputs)
    probs = torch.nn.functional.softmax(outputs.logits, dim = -1).tolist()[0]


    if probs[2] > probs[0]:
        return 'positive'
    else:
        return 'negative'

merged_df["Sentiment"] = merged_df["Article_title"].apply(get_sent)


print(merged_df[['Article_title', 'Sentiment']].head())
